In [1]:
import numpy as np  # linear algebra
from matplotlib import pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from models.EnsembleModel import EnsembleModel
from models.InceptionV4 import InceptionV4
from models.MobileNet import MobileNet
from models.XCeption import XCeption
from models.Resnet50 import Resnet50
from models.VGG16_vggface import VGG16_VGGFACE
from models.Resnet50_vggface import Resnet50_VGGFACE

import tensorflow as tf



In [2]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.InteractiveSession(config=config)

In [3]:
def train_model(model, name,train_data,val_data):
    epochs = 15

    history = model.train(
        epochs=epochs,
        train_dataset=train_data,
        validation_dataset=val_data,
    )
kl
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(name + ' model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(name + ' model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()


In [4]:
datagen = ImageDataGenerator(
    rescale=1 / 255,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.3,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest',
    validation_split=0.1
)

datagen_dev = ImageDataGenerator(
    rescale=1 / 255,
)

datagen_test = ImageDataGenerator(
    rescale=1 / 255,
)

target_size = 197

train = datagen.flow_from_directory(
    'data/train',
    target_size=(target_size, target_size),
    batch_size=16,
    subset='training',
    class_mode='categorical',
    interpolation='hamming'
)

val = datagen.flow_from_directory(
    'data/train',
    target_size=(target_size, target_size),
    batch_size=16,
    subset='validation',
    class_mode='categorical',
    interpolation='hamming'
)

test = datagen_test.flow_from_directory(
    'data/dev',
    target_size=(197, 197),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 30336 images belonging to 7 classes.
Found 3369 images belonging to 7 classes.
Found 3619 images belonging to 7 classes.


In [5]:
def generate_generator_multiple(generator, subset):
    genX1 = generator.flow_from_directory(
        'data/train',
        target_size=(target_size, target_size),
        batch_size=16,
        subset=subset,
        class_mode='categorical',
        interpolation='hamming',
        seed=7
    )

    genX2 = generator.flow_from_directory(
        'data/train',
        target_size=(target_size, target_size),
        batch_size=16,
        subset=subset,
        class_mode='categorical',
        interpolation='hamming',
        seed=7
    )
    while True:
        X1i = genX1.next()
        X2i = genX2.next()
        yield [X1i[0], X2i[0]], X2i[1]  # Yield both images and their mutual label

In [6]:
def generate_test_generator_multiple(generator):
    genX1 = generator.flow_from_directory(
        'data/dev',
        target_size=(target_size, target_size),
        batch_size=16,
        class_mode='categorical',
        interpolation='hamming',
        seed=7
    )

    genX2 = generator.flow_from_directory(
        'data/dev',
        target_size=(target_size, target_size),
        batch_size=16,
        class_mode='categorical',
        interpolation='hamming',
        seed=7
    )
    while True:
        X1i = genX1.next()
        X2i = genX2.next()
        yield [X1i[0], X2i[0]], X2i[1]  # Yield both images and their mutual label

In [7]:
train_generator = generate_generator_multiple(datagen, 'training')
validation_generator = generate_generator_multiple(datagen, 'validation')
test_generator = generate_test_generator_multiple(datagen_dev)

In [ ]:
mymodel = EnsembleModel(target_size)
print(mymodel.model.summary())
train_model(mymodel, "Ensemble",train_generator,validation_generator)



mymodel.model.evaluate(test_generator, steps=len(test.filenames) / 16)

197
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 197, 197, 3)]     0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 197, 197, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 197, 197, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 98, 98, 64)        0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 98, 98, 128)       73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 98, 98, 128)       147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 49, 49, 128)       

Found 30336 images belonging to 7 classes.
Found 30336 images belonging to 7 classes.
Epoch 1/15


  92/1896 [>.............................] - ETA: 6:26 - loss: 1.7945 - accuracy: 0.4708

In [ ]:
#Use this cell after cells below!
mymodel.model.evaluate(test)

In [8]:
mymodel = VGG16_VGGFACE(target_size)  # %95 train accuracy , %65 test accuracy
train_model(mymodel, 'VGGFACE-VGG16 Model',train,val)

197
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 197, 197, 3)]     0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 197, 197, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 197, 197, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 98, 98, 64)        0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 98, 98, 128)       73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 98, 98, 128)       147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 49, 49, 128)       0 

KeyboardInterrupt: 

In [ ]:
mymodel = InceptionV4(target_size)  # %95 train accuracy , %70 test accuracy
train_model(mymodel, 'Inception Model')



In [ ]:
mymodel = Resnet50_VGGFACE(target_size)
train_model(mymodel, 'VGGFACE-RESNET50')

In [ ]:
mymodel = MobileNet(target_size)  # %95 train accuracy , %64 test accuracy
train_model(mymodel, 'MobileNet Model')


In [ ]:
mymodel = Resnet50(target_size)  # %95 train accuracy , %64 test accuracy
train_model(mymodel, 'Resnet Model')



In [ ]:
mymodel = XCeption(target_size)
train_model(mymodel, 'Xception Model')
